In [ ]:
import ray
ray.init()
# ray.shutdown()

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)


import pandas_bokeh
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import polygon_ds as pds
import polygon_df as pdf
import polygon_s3 as ps3
import ray_symbol_details as rsd
import market_preprocess as mp
import market_cov as mcov
import market_cluster as mc
import utils_pickle as up
# import ipdb
# ipdb.set_trace(context=10)

In [2]:
start_date = '2018-10-24'
end_date = '2020-11-24'
symbol = 'market'
tick_type='daily'

In [3]:
r = mp.prepare_data(start_date, end_date, beta_symbol='VTI')

4502494 Initial rows 11509 symbols
-1460110 all dates filter 5784 symbols
-383980 liquidity filter 5054 symbols
-125188 volitility filter 4816 symbols
-24196 min $value filter 4770 symbols
-706944 symbol details filter 3426 symbols
1802076 Final rows 40.0 % remaining


100%|██████████| 3426/3426 [00:25<00:00, 136.27it/s]


In [5]:
X = r['g_zs_log_returns_resid']
Xt = X.transpose()
gt_industry = r['sym_meta'].industry.values
gt_sector = r['sym_meta'].sector.values

In [6]:
# r['g_zs_log_returns_resid'].iloc[:, 1500:]

dtdist_mat, dtcor_mat = mcov.cov_denoise_detone_dist(r['g_zs_log_returns'], detone=True)

dist_mat, cor_mat = mcov.cov_denoise_detone_dist(r['g_zs_log_returns_resid'], detone=False)

In [ ]:
kclust = mc.cluster_ground_truth_eval(mc.get_kmean_clusters, 200, X, gt_industry)
print(kclust['adj. multual information'])

In [7]:
hclust = mc.cluster_ground_truth_eval(mc.get_hira_clusters, 200, Xt, gt_industry)
print(hclust['adj. multual information'])

0.4304010633801613


In [16]:
labels = pd.Series(hclust['cluster_labels'], name='lables')

In [26]:
out = mc.cluster_metrics(lables=labels, dist_mat=dist_mat, sym_meta=r['sym_meta'])

In [32]:
out

,size,avg_similartiy,sector_purity,industry_purity,avg_range_value_pct,med_daily_dollar_volume,dist_mat,symbol_meta
label,,,,,,,,
0,45,0.371480,0.200000,0.062500,0.028590,4532413.000000,symbol APOG ASGN AVD B ...,...
1,7,0.324171,0.333333,0.250000,0.060612,253612.062500,symbol AUTO BBGI CLSN EVC ...,name...
2,20,0.355337,0.500000,0.333333,0.018622,107984336.000000,symbol AJG AON BRO CBOE ...,...
3,12,0.335492,1.000000,0.500000,0.074794,474621.812500,symbol AVDL CKPT FBIO LIFE ...,name sect...
4,23,0.320709,0.333333,0.142857,0.043901,2546967.000000,symbol AAOI ACRS AUDC AVID ...,...
...,...,...,...,...,...,...,...,...
195,35,0.382033,0.250000,0.100000,0.029035,27157964.000000,symbol ACCO ACM AIMC ASH ...,...
196,25,0.440634,1.000000,0.500000,0.030825,13579351.000000,symbol BPMP CEQP CQP DCP ...,...
197,6,0.336181,0.333333,0.200000,0.044761,6309849.000000,symbol CPSI CUTR EHTH HBIO ...,name ...


In [ ]:
mc.gt_cluster_eval(kclust['cluster_labels'], hclust['cluster_labels'])

In [ ]:
kmeadist_mat                             hclust
-------------------------          ----------------------------
X norn
X nrom resids: 45                   X norm resids: 464
X resids dncor dist: 34
X resids corr: 33
X resids dncor detone: 37           X resids dncor detone: 386

In [ ]:
corex300 = up.pickle_load('data/corex300.pkl')

# X = df.reset_index(drop=True)
# corex, full_df = mc.corex_fit(X, n_hidden=200)

# up.pickle_dump((corex, full_df), file_name='corex200.pkl')

# gsutil copy /Users/bobcolner/QuantClarity/pandas-polygon/corex200.pkl gs://emerald-skill-datascience/

In [ ]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator
h2o.init()

# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/pca_test/USArrests.csv")

# Split the dataset into a train and valid set:
train, valid = arrestsH2O.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(
    k=4,
    loss="Huber",
    loss_by_col={'cat_var': 'Categorical'},
    regularization_x="OneSparse", # None (default), Quadratic, L2, L1, NonNegative, OneSparse, UnitOneSparse, Simplex.
    regularization_y="OneSparse",
    gamma_x=0.5,
    gamma_y=0.5,
    max_iterations=1000,
    recover_svd=True,
    init="PlusPlus",
    transform="standardize",
   )
glrm_model.train(training_frame=train)